# Player Direction Prediction - Clean Training
### Soccer Player Body Orientation Prediction using ResNet34

This notebook trains a deep learning model to predict player body direction (0-360°) from images.

## 1. Mount Google Drive

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Install Dependencies

In [12]:
!pip install -q torch torchvision pillow tqdm

## 3. Imports

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from pathlib import Path
import json
import math
import numpy as np
from PIL import Image
from tqdm import tqdm
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

✅ Using device: cuda


## 4. Configuration

In [14]:
# ============================================================
# CORRECTED CONFIGURATION
# ============================================================
CONFIG = {
    # Paths - CORRECTED
    'data_root': '/content/drive/MyDrive/player_direction_dataset',
    'save_dir': '/content/drive/MyDrive/player_direction_models',
    'experiment_name': 'resnet34_clean_v1',

    # Model
    'backbone': 'resnet34',

    # Training
    'batch_size': 32,
    'num_epochs': 200,
    'learning_rate': 5e-5,
    'weight_decay': 1e-3,

    # Early stopping
    'early_stopping_patience': 30,
    'early_stopping_min_delta': 0.5,

    # Data
    'num_workers': 2,
    'pin_memory': True,
}

print("✅ Configuration loaded")

✅ Configuration loaded


## 5. Dataset Class

In [15]:
class PlayerDirectionDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        if split == 'val':
            split = 'valid'

        self.root_dir = Path(root_dir) / split
        self.transform = transform

        labels_file = self.root_dir / 'labels.json'
        with open(labels_file, 'r') as f:
            labels_list = json.load(f)

        self.labels = {item['filename']: item['direction_degree'] for item in labels_list}
        self.image_files = list(self.labels.keys())

        print(f"✅ {split} set: {len(self.image_files)} images")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = self.root_dir / 'images' / img_name

        image = Image.open(img_path).convert('RGB')
        angle_deg = self.labels[img_name]

        if self.transform:
            image = self.transform(image)

        angle_rad = math.radians(angle_deg)

        return {
            'image': image,
            'sin': torch.tensor(math.sin(angle_rad), dtype=torch.float32),
            'cos': torch.tensor(math.cos(angle_rad), dtype=torch.float32)
        }

## 6. Data Transforms

In [16]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ Transforms defined")

✅ Transforms defined


## 7. Model Architecture

In [17]:
class PlayerDirectionPredictor(nn.Module):
    def __init__(self):
        super().__init__()

        # Load pretrained ResNet34
        resnet = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(*list(resnet.children())[:-1])

        # Prediction head
        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Output: (sin, cos)
        )

    def forward(self, x):
        features = self.features(x)
        sin_cos = self.head(features)
        # Normalize to unit circle
        sin_cos = nn.functional.normalize(sin_cos, p=2, dim=1)
        return sin_cos

    def predict_angle(self, x):
        sin_cos = self.forward(x)
        angle_rad = torch.atan2(sin_cos[:, 0], sin_cos[:, 1])
        angle_deg = torch.rad2deg(angle_rad) % 360
        return angle_deg

print("✅ Model architecture defined")

✅ Model architecture defined


## 8. Loss Function

In [18]:
class CircularLoss(nn.Module):
    def forward(self, pred, target_sin, target_cos):
        pred_sin, pred_cos = pred[:, 0], pred[:, 1]
        loss = 1 - (pred_sin * target_sin + pred_cos * target_cos)
        return loss.mean()

print("✅ Loss function defined")

✅ Loss function defined


## 9. Training & Validation Functions

In [19]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_mae = 0

    pbar = tqdm(loader, desc="Training")
    for batch in pbar:
        images = batch['image'].to(device)
        target_sin = batch['sin'].to(device)
        target_cos = batch['cos'].to(device)

        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, target_sin, target_cos)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Calculate MAE
        with torch.no_grad():
            pred_angle = model.predict_angle(images)
            target_angle = torch.rad2deg(torch.atan2(target_sin, target_cos)) % 360
            mae = torch.mean(torch.abs(pred_angle - target_angle))

        total_loss += loss.item()
        total_mae += mae.item()

        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'mae': f'{mae.item():.2f}°'})

    return total_loss / len(loader), total_mae / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    total_mae = 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validating")
        for batch in pbar:
            images = batch['image'].to(device)
            target_sin = batch['sin'].to(device)
            target_cos = batch['cos'].to(device)

            pred = model(images)
            loss = criterion(pred, target_sin, target_cos)

            pred_angle = model.predict_angle(images)
            target_angle = torch.rad2deg(torch.atan2(target_sin, target_cos)) % 360
            mae = torch.mean(torch.abs(pred_angle - target_angle))

            total_loss += loss.item()
            total_mae += mae.item()

            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'mae': f'{mae.item():.2f}°'})

    return total_loss / len(loader), total_mae / len(loader)

print("✅ Training functions defined")

✅ Training functions defined


## 10. Load Data

In [20]:
print("\n" + "="*70)
print("📂 LOADING DATASETS")
print("="*70)

train_dataset = PlayerDirectionDataset(CONFIG['data_root'], 'train', train_transform)
val_dataset = PlayerDirectionDataset(CONFIG['data_root'], 'val', val_transform)
test_dataset = PlayerDirectionDataset(CONFIG['data_root'], 'test', val_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory']
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory']
)

test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=CONFIG['pin_memory']
)

print(f"✅ Data loaders ready")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")
print(f"   Test batches: {len(test_loader)}")


📂 LOADING DATASETS
✅ train set: 1263 images
✅ valid set: 270 images
✅ test set: 272 images
✅ Data loaders ready
   Train batches: 40
   Val batches: 9
   Test batches: 9


## 11. Initialize Training

In [21]:
print("\n" + "="*70)
print("🔧 INITIALIZING TRAINING")
print("="*70)

model = PlayerDirectionPredictor().to(device)
criterion = CircularLoss()
optimizer = optim.AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay']
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=10,
    min_lr=1e-7
)

# Create save directory
save_dir = Path(CONFIG['save_dir']) / CONFIG['experiment_name']
save_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Model: {CONFIG['backbone']}")
print(f"✅ Optimizer: AdamW (lr={CONFIG['learning_rate']}, wd={CONFIG['weight_decay']})")
print(f"✅ Scheduler: ReduceLROnPlateau")
print(f"✅ Save directory: {save_dir}")


🔧 INITIALIZING TRAINING
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 172MB/s]


✅ Model: resnet34
✅ Optimizer: AdamW (lr=5e-05, wd=0.001)
✅ Scheduler: ReduceLROnPlateau
✅ Save directory: /content/drive/MyDrive/player_direction_models/resnet34_clean_v1


## 12. Training Loop

In [22]:
print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)
print(f"Max Epochs: {CONFIG['num_epochs']}")
print(f"Early Stopping: {CONFIG['early_stopping_patience']} epochs")
print(f"Batch Size: {CONFIG['batch_size']}")
print("="*70)

best_val_mae = float('inf')
patience_counter = 0
start_time = time.time()
history = []

for epoch in range(CONFIG['num_epochs']):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{CONFIG['num_epochs']}")
    print(f"{'='*70}")

    # Train
    train_loss, train_mae = train_epoch(model, train_loader, criterion, optimizer, device)

    # Validate
    val_loss, val_mae = validate(model, val_loader, criterion, device)

    # Update scheduler
    scheduler.step(val_mae)

    # Log results
    current_lr = optimizer.param_groups[0]['lr']
    print(f"\n📊 Epoch {epoch+1} Summary:")
    print(f"   Train - Loss: {train_loss:.4f}, MAE: {train_mae:.2f}°")
    print(f"   Valid - Loss: {val_loss:.4f}, MAE: {val_mae:.2f}°")
    print(f"   LR: {current_lr:.6f}")

    # Save history
    history.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_mae': train_mae,
        'val_loss': val_loss,
        'val_mae': val_mae,
        'lr': current_lr
    })

    # Save best model
    if val_mae < best_val_mae - CONFIG['early_stopping_min_delta']:
        best_val_mae = val_mae
        patience_counter = 0

        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_mae': val_mae,
            'config': CONFIG
        }
        torch.save(checkpoint, save_dir / 'best_model.pth')
        print(f"   💾 Best model saved! (MAE: {val_mae:.2f}°)")
    else:
        patience_counter += 1
        print(f"   ⚠️  No improvement for {patience_counter}/{CONFIG['early_stopping_patience']} epochs")

    # Early stopping
    if patience_counter >= CONFIG['early_stopping_patience']:
        print(f"\n🛑 EARLY STOPPING TRIGGERED!")
        print(f"   No improvement for {CONFIG['early_stopping_patience']} epochs")
        print(f"   Best epoch: {epoch + 1 - patience_counter} (MAE: {best_val_mae:.2f}°)")
        break

training_time = (time.time() - start_time) / 3600

print("\n" + "="*70)
print("✅ TRAINING COMPLETED")
print("="*70)
print(f"Total Time: {training_time:.2f} hours")
print(f"Best Validation MAE: {best_val_mae:.2f}°")
print(f"Model saved to: {save_dir / 'best_model.pth'}")
print("="*70)

# Save training history
with open(save_dir / 'training_history.json', 'w') as f:
    json.dump(history, f, indent=2)
print(f"📈 Training history saved to: {save_dir / 'training_history.json'}")


🚀 STARTING TRAINING
Max Epochs: 200
Early Stopping: 30 epochs
Batch Size: 32

Epoch 1/200


Validating: 100%|██████████| 9/9 [00:44<00:00,  4.99s/it, loss=0.6202, mae=61.59°]



📊 Epoch 1 Summary:
   Train - Loss: 0.9712, MAE: 104.95°
   Valid - Loss: 0.9313, MAE: 99.61°
   LR: 0.000050
   💾 Best model saved! (MAE: 99.61°)

Epoch 2/200


Validating: 100%|██████████| 9/9 [00:01<00:00,  5.66it/s, loss=0.6513, mae=68.25°]



📊 Epoch 2 Summary:
   Train - Loss: 0.8824, MAE: 99.57°
   Valid - Loss: 0.7581, MAE: 94.51°
   LR: 0.000050
   💾 Best model saved! (MAE: 94.51°)

Epoch 3/200


...


📊 Epoch 78 Summary:
   Train - Loss: 0.0172, MAE: 18.28°
   Valid - Loss: 0.4112, MAE: 85.16°
   LR: 0.000002
   ⚠️  No improvement for 28/30 epochs

Epoch 79/200


Validating: 100%|██████████| 9/9 [00:02<00:00,  4.27it/s, loss=0.2293, mae=52.20°]



📊 Epoch 79 Summary:
   Train - Loss: 0.0170, MAE: 18.21°
   Valid - Loss: 0.3903, MAE: 79.14°
   LR: 0.000002
   ⚠️  No improvement for 29/30 epochs

Epoch 80/200


Validating: 100%|██████████| 9/9 [00:01<00:00,  5.07it/s, loss=0.2520, mae=49.56°]


📊 Epoch 80 Summary:
   Train - Loss: 0.0191, MAE: 19.74°
   Valid - Loss: 0.4333, MAE: 90.98°
   LR: 0.000002
   ⚠️  No improvement for 30/30 epochs

🛑 EARLY STOPPING TRIGGERED!
   No improvement for 30 epochs
   Best epoch: 50 (MAE: 73.26°)

✅ TRAINING COMPLETED
Total Time: 0.37 hours
Best Validation MAE: 73.26°
Model saved to: /content/drive/MyDrive/player_direction_models/resnet34_clean_v1/best_model.pth
📈 Training history saved to: /content/drive/MyDrive/player_direction_models/resnet34_clean_v1/training_history.json


## 13. Test Evaluation

In [23]:
print("\n" + "="*70)
print("📊 EVALUATING ON TEST SET")
print("="*70)

# Load best model
checkpoint = torch.load(save_dir / 'best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Loaded best model from epoch {checkpoint['epoch']}")

# Evaluate
test_loss, test_mae = validate(model, test_loader, criterion, device)

print(f"\n{'='*70}")
print("📊 TEST SET RESULTS")
print(f"{'='*70}")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test MAE: {test_mae:.2f}°")
print(f"{'='*70}")

# Save test results
test_results = {
    'test_loss': test_loss,
    'test_mae': test_mae,
    'best_epoch': checkpoint['epoch'],
    'best_val_mae': best_val_mae
}

with open(save_dir / 'test_results.json', 'w') as f:
    json.dump(test_results, f, indent=2)

print(f"\n✅ Test results saved to: {save_dir / 'test_results.json'}")

print("\n" + "="*70)
print("🎉 ALL DONE!")
print("="*70)


📊 EVALUATING ON TEST SET
✅ Loaded best model from epoch 50


Validating: 100%|██████████| 9/9 [00:47<00:00,  5.28s/it, loss=0.6061, mae=131.79°]


📊 TEST SET RESULTS
Test Loss: 0.4520
Test MAE: 86.46°

✅ Test results saved to: /content/drive/MyDrive/player_direction_models/resnet34_clean_v1/test_results.json

🎉 ALL DONE!
